In [ ]:
import os
os.environ["PYTORCH_JIT_USE_NNC_NOT_NVFUSER"] = "1"
from typing import List, Tuple, Optional, Union, Iterable
import warnings
import math

from beartype import beartype
import datetime
import plotly.graph_objects as go
from tqdm import tqdm
import yaml

import torch
from torch.utils.data import DataLoader
from torchvision.transforms import Compose
from e3nn import o3

from edf_interface.data import PointCloud, SE3, DemoDataset, TargetPoseDemo
from edf_interface.utils import manipulation_utils
from diffusion_edf.gnn_data import FeaturedPoints
from diffusion_edf import train_utils
from diffusion_edf.trainer import DiffusionEdfTrainer
from diffusion_edf.visualize import visualize_pose
from diffusion_edf.agent import DiffusionEdfAgent

torch.set_printoptions(precision=4, sci_mode=False)

In [ ]:
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# COPY AND PASTE THE NOTEBOOK TO PROJECT ROOT DIR (WITH 'config' AND 'demo' DIR) BEFORE RUNNING IT
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

device = 'cuda:0'
task_type = 'place'
config_root_dir = 'configs/panda_mug'
testset = DemoDataset(dataset_dir='demo/example_demo_real_mug_franka_panda')

In [ ]:
with open(os.path.join(config_root_dir, 'agent.yaml')) as f:
    model_kwargs = yaml.load(f, Loader=yaml.FullLoader)
    model_kwargs_list = model_kwargs['model_kwargs'][f"{task_type}_models_kwargs"]
    critic_kwargs = model_kwargs['model_kwargs'][f"{task_type}_critic_kwargs"]

with open(os.path.join(config_root_dir, 'preprocess.yaml')) as f:
    preprocess_config = yaml.load(f, Loader=yaml.FullLoader)
    unprocess_config = preprocess_config['unprocess_config']
    preprocess_config = preprocess_config['preprocess_config']

agent = DiffusionEdfAgent(
    model_kwargs_list=model_kwargs_list,
    preprocess_config=preprocess_config,
    unprocess_config=unprocess_config,
    device=device,
    critic_kwargs=critic_kwargs
) # Model initialization and warm-up takes 2~3 minutes (very slow due to e3nn codegen and torch jit).

# Initialize Input Data and Initial Pose

In [ ]:
demo: TargetPoseDemo = testset[1][0 if task_type == 'pick' else 1 if task_type == 'place' else "task_type must be either 'pick' or 'place'"].to(device)
scene_pcd: PointCloud = demo.scene_pcd
grasp_pcd: PointCloud = demo.grasp_pcd

In [ ]:
N_samples = 10 # reduce number of samples if too slow
T0 = torch.cat([
    torch.tensor([[1., 0., 0.0, 0.]], device=device),
    torch.tensor([[0., 0., 0.3]], device=device)
], dim=-1).repeat(N_samples, 1)
Ts_init = SE3(poses=T0).to(device)


Ts_out_raw, scene_proc, grasp_proc, info = agent.sample(scene_pcd=scene_pcd, grasp_pcd=grasp_pcd, Ts_init=Ts_init,
                                                        N_steps_list = [[200, 200], [200, 200]],
                                                        timesteps_list = [[0.04, 0.04], [0.02, 0.02]],
                                                        temperatures_list = [[1., 1.], [1., 1.]],
                                                        log_t_schedule = True,
                                                        diffusion_schedules_list = [
                                                            [[1., 0.1], [0.1, 0.1]],
                                                            [[0.1, 0.03], [0.03, 0.01] ],
                                                        ],
                                                        time_exponent_temp = 1.0,
                                                        time_exponent_alpha = 0.5,
                                                        return_info=True)

# Ts_out, energy = Ts_out_raw, info['energy']
Ts_out, energy = manipulation_utils.remove_outliers(Ts_out_raw.permute(1,0,2), critic=info['energy'])
Ts_out = torch.stack(Ts_out, dim=0).permute(1,0,2)

In [ ]:
sample_idx = 0
visualization = TargetPoseDemo(
    target_poses=SE3(poses=torch.cat([Ts_out[::10, sample_idx], Ts_out[-1:, sample_idx]], dim=0)),
    scene_pcd=scene_proc,
    grasp_pcd=grasp_proc
)
visualization = agent.unprocess_fn(visualization).to('cpu')
visualization.show()